In [1]:
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
import tensorflow as tf

In [3]:
def create_pretrained_model(base_model):
    base_model.trainable = False
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(512, activation='relu'),
        Dense(1, activation='sigmoid')  #
    ])
    return model

In [4]:
resnet_base = ResNet50(input_shape=(150, 150, 3), include_top=False, weights='imagenet')
resnet_model = create_pretrained_model(resnet_base)

In [5]:
resnet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print("Training ResNet50 model...")
resnet_history = resnet_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    verbose=1
)

Training ResNet50 model...


NameError: name 'train_gen' is not defined

In [ ]:
resnet_loss, resnet_accuracy = resnet_model.evaluate(val_gen)
print(f"ResNet50 Test Loss: {resnet_loss}")
print(f"ResNet50 Test Accuracy: {resnet_accuracy}")

In [ ]:
vgg_base = VGG16(input_shape=(150, 150, 3), include_top=False, weights='imagenet')
vgg_model = create_pretrained_model(vgg_base)

In [ ]:
vgg_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print("Training VGG16 model...")
vgg_history = vgg_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    verbose=1
)

In [ ]:
vgg_loss, vgg_accuracy = vgg_model.evaluate(val_gen)
print(f"VGG16 Test Loss: {vgg_loss}")
print(f"VGG16 Test Accuracy: {vgg_accuracy}")


In [ ]:
import matplotlib.pyplot as plt


def plot_metrics(history1, history2, model1_name='ResNet50', model2_name='VGG16'):

    plt.figure(figsize=(14, 5))

    plt.subplot(1, 2, 1)
    plt.plot(history1.history['accuracy'], label=f'{model1_name} Train Accuracy')
    plt.plot(history1.history['val_accuracy'], label=f'{model1_name} Val Accuracy')
    plt.plot(history2.history['accuracy'], label=f'{model2_name} Train Accuracy')
    plt.plot(history2.history['val_accuracy'], label=f'{model2_name} Val Accuracy')
    plt.title('Model Accuracy Comparison')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(loc="upper left")


    plt.subplot(1, 2, 2)
    plt.plot(history1.history['loss'], label=f'{model1_name} Train Loss')
    plt.plot(history1.history['val_loss'], label=f'{model1_name} Val Loss')
    plt.plot(history2.history['loss'], label=f'{model2_name} Train Loss')
    plt.plot(history2.history['val_loss'], label=f'{model2_name} Val Loss')
    plt.title('Model Loss Comparison')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(loc="upper left")

    plt.show()


plot_metrics(resnet_history, vgg_history, model1_name='ResNet50', model2_name='VGG16')


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np


def evaluate_model(model, generator, model_name='Model'):

    y_true = generator.classes
    y_pred = np.round(model.predict(generator)).astype(int)


    cm = confusion_matrix(y_true, y_pred)
    print(f"Confusion Matrix for {model_name}:")
    print(cm)


    report = classification_report(y_true, y_pred, target_names=generator.class_indices.keys())
    print(f"\nClassification Report for {model_name}:\n{report}")


evaluate_model(resnet_model, val_gen, model_name='ResNet50')
evaluate_model(vgg_model, val_gen, model_name='VGG16')
